In [1]:

# Authenticates and initializes Earth Engine
import ee
ee.Authenticate()

ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=cHJhIlmpaI-Ylc854sNthl2uiU_KDi-nLJ-GXiy5Efo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgeBYy3R_j_narjtH_33DrZtxADKRHm4gAh_qVGqXEA5aIzjEZwIjE

Successfully saved authorization token.


In [2]:
##tiles = ["44QLG", "44QLE", "44QLD", "44QKE", "44PKC", "44QKG", "44QKD", "44QMF", "44QKF", "44QME", "44QMD"]
##tiles = ["43PGT","43PHT","43QGA","43QGB","43QGU","43QGV","43QHA","43QHB","43QHC","43QHU","43QHV","44PKC","44PLC","44QKD","44QKE","44QKF","44QKG","44QKH","44QLD","44QLE","44QLF","44QLG","44QMD","44QME","44QMF","44QMG","44QND","44QNE","44QNF"]
tiles = ["44QKG","44QKF","44QLE", "44QLF"]
#tiles = ["44QLD"]
##print(sentinel2_tile_bound)
startDate = '2021-10-20'
endDate = '2021-10-21'
## function for cloud masking
def maskCloudAndShadows(image):
  cloudProb = image.select('MSK_CLDPRB')
  snowProb = image.select('MSK_SNWPRB')
  cloud = cloudProb.lt(5)
  snow = snowProb.lt(5)
  scl = image.select('SCL')
  shadow = scl.eq(3) ## 3 = cloud shadow
  cirrus = scl.eq(10) ## 10 = cirrus
  ## Cloud probability less than 5% or cloud shadow classification
  mask = (cloud.And(snow)).And(cirrus.neq(1)).And(shadow.neq(1))
  return image.updateMask(mask)
for tile in tiles:
  sentinel2_tile_bound = ee.FeatureCollection('users/saptarshi/SRSP_Sentinel2_boundary').filter(ee.Filter.eq('Name', tile))
  #bound = ee.FeatureCollection('users/saptarshi/USA_geometry')
  #sentinel2_tile_bound = ee.FeatureCollection('users/saptarshi/Telangana_Sentinel2_tile_boundary_10kmbuffer').filter(ee.Filter.eq('Name', tile))
  ## Define an example collection.
  collection = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(startDate, endDate).filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 80)).filter(ee.Filter.eq('MGRS_TILE', tile))
  #collection = ee.ImageCollection('COPERNICUS/S2_SR').filterDate(startDate, endDate).filter(ee.Filter.lt('CLOUD_COVERAGE_ASSESSMENT', 20)).map(maskCloudAndShadows).filter(ee.Filter.eq('MGRS_TILE', tile))

  ##NDVIcollection = collection.map(calculateNDVI)
  count = int(collection.size().getInfo())
  print("Total number of images: {}".format(count))
  img_lst = collection.toList(count)
  ##print(img_lst)
  for i in range(0, count):
      image1 = ee.Image(collection.toList(count).get(i))
      region = sentinel2_tile_bound.geometry() #image.geometry() ## roi
      #region = image1.geometry() ## roi
      dateString = image1.get('system:index').getInfo()
      #ndvi = image.normalizedDifference(['B8', 'B4'])
      #ndvi2 = ndvi.multiply(10000).toInt16()
      #tci_R = image1.select('TCI_R')
      #tci_G = image1.select('TCI_G')
      #tci_B = image1.select('TCI_B')
      tci_B3 = image1.select('B3')
      tci_B4 = image1.select('B4')
      tci_B5 = image1.select('B5')
      tci_B6 = image1.select('B6')
      tci_B7 = image1.select('B7')
      tci_B8 = image1.select('B8')
      tci_B8A = image1.select('B8A')
      tci_B11 = image1.select('B11')
      tci_B12 = image1.select('B12')

      tci = (tci_B3.addBands(ee.Image(tci_B4)).addBands(ee.Image(tci_B5)).addBands(ee.Image(tci_B6)).addBands(ee.Image(tci_B7)).addBands(ee.Image(tci_B8)).addBands(ee.Image(tci_B8A))
              .addBands(ee.Image(tci_B11)).addBands(ee.Image(tci_B12)))
      #print(tci.getInfo()['bands'])
      #tci = tci_R.addBands(ee.Image(tci_G)).addBands(ee.Image(tci_B))
      name = "Seninel2_with_cloud_T" + tile + "_" + dateString[:-34] + "-" + dateString[-34:-32] + "-" + dateString[-32:-30] # set the output image filename
      #name = "Seninel2_with_cloud_TCI_T" + tile + "_" + dateString[:-34] + "-" + dateString[-34:-32] + "-" + dateString[-32:-30]
      #print(name)
      # Set configration parameters for output image
      task_config = {
      ##'folder': 'Sentinel2-NDVI_data', # output Google Drive folder
      'region': region,     # roi 
      'scale': 10,       # image resolution
      'crs': 'EPSG:4326',
      'maxPixels': 1.0E13,
      'fileFormat': 'GeoTIFF'
      }
      task = ee.batch.Export.image(tci.clip(region), name, task_config)
      #task = ee.batch.Export.image(ndvi2.clip(region), name, task_config)
      task.start()
      print (name + " is exported")
  print("Process over for tile no: " + tile)

Total number of images: 2
Seninel2_with_cloud_T44QKG_2021-10-20 is exported
Seninel2_with_cloud_T44QKG_2021-10-20 is exported
Process over for tile no: 44QKG
Total number of images: 0
Process over for tile no: 44QKF
Total number of images: 0
Process over for tile no: 44QLE
Total number of images: 0
Process over for tile no: 44QLF
